In [ ]:
"""
This script generates nifti files from the parseEPI-outputted EPI time series .mat files
(ending in "_interp.mat") from the listening and reading control tasks. The purpose of
this step is to provide easy-to-use inputs for generating pymvpa datasets that can then
be used for hyperalignment (or other pymvpa friendly things like decoding).
"""

In [ ]:
import numpy as np
import glob
from nilearn import image as nImage
import nibabel as nb
from nilearn import input_data
import scipy.io as sio
from scipy import stats

In [ ]:
# select hyperscanning task
# 3: listening
# 4: reading
task = 4

# Select whether or not to generate two short nifti files for the purposes of debugging
# hyperalignment scripts. If false, we will glob all relevant .mat files for the selected
# hyperscanning task. If True, the selected hyperscanning task (above) is irrelevant.
debug = False

In [ ]:
# output path and debugging output string tag
if debug:
    saveFolder = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/preprocessing/hyperalignment/input_nifti_files/debugging/'
    debugStr = '_debug'
else:
    saveFolder = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/preprocessing/hyperalignment/input_nifti_files/'
    debugStr = ''

# set base path and search terms (nuisRegr .nii output first, parseEPI .mat output second)
searchTerms = [[]] * 2
basePath = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/control_tasks/nuisRegr_output_files/'
searchTerms[0] = '*storytelling' + str(task) + '_run-0' + str(task) + '_bold_space-MNI152NLin2009cAsym_preproc_nuisRegr_newMask.nii.gz'
searchTerms[1] = '*storytelling' + str(task) + '_run-0' + str(task) + '_bold_space-MNI152NLin2009cAsym_preproc_nuisRegr_newMask_interp.mat'

# glob files
files = [[]] * 2
for TERM in range(len(searchTerms)):
    files[TERM] = glob.glob(basePath + searchTerms[TERM], recursive=True)

# get corresponding .nii and .mat pairs
filePairs = []
for NIIFILE in range(len(files[0])):

    # get portion of current nifti file path that identifies participant and task
    filePath = files[0][NIIFILE]
    subStrStart = filePath.find('sub-')
    subStrEnd = filePath.find('storytelling' + str(task)) + len('storytelling' + str(task))
    subStr = filePath[subStrStart:subStrEnd]

    # look for a matching string in the .mat files
    for MATFILE in range(len(files[1])):

        if subStr in files[1][MATFILE]:
            filePairs.append([files[0][NIIFILE],files[1][MATFILE]])

# truncate the pairs list to just the first two if we're debugging
if debug:
    filePairs = filePairs[:2]

In [ ]:
resampledMaskFile = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/misc/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_mask_RESAMPLED.nii'
maskImg = nImage.load_img(resampledMaskFile)

In [ ]:
# for each subject with a pair of .nii and /mat files for the selected task...
for SUB in range(len(filePairs)):

    # initialize masker object from whole brain mask and nuisRegr output .nii file
    masker = input_data.NiftiMasker(maskImg)
    masker.fit_transform(filePairs[SUB][0])

    # load parsed EPI time series
    tmp = sio.loadmat(filePairs[SUB][1]) #load file
    parsedEPI = tmp['tseries'] #get timeseries data

    # truncate time series if this is for debugging
    if debug:
        parsedEPI = parsedEPI[0:40,:]

    # standardize the time series
    parsedEPI = stats.zscore(parsedEPI,axis=0)

    #%% make new nifti with parsedEPI time series
    outputFile = saveFolder + filePairs[SUB][1][subStrStart:-4] + debugStr + '.nii.gz'
    print('saving file ' + str(SUB + 1) + ' of ' + str(len(filePairs)) + ' to: ')
    print(outputFile)
    cleaned_img = masker.inverse_transform(parsedEPI)
    cleaned_img.to_filename(outputFile)